In [3]:
import numpy as np
import scipy.stats as sstats
from matplotlib import pyplot as plt
import ipywidgets as wid

In [4]:
def linmat(alpha=1, beta=1, M=1): # underdetermined 2 to 1 
    Q_map = np.array( [[alpha, beta]] )
    Q_map = np.tile(Q_map.T, M).T
    return Q_map

def gauss_sol(prior_mean, prior_std, data_std, A, data): # DO NOTE THAT SIZES HERE ARE Column-Major instead of Row-Major ... (dim, samps)
    if type(prior_mean) is int:
        prior_mean = [prior_mean, prior_mean]
    if type(prior_mean) is float:
        prior_mean = [prior_mean, prior_mean]
    if type(prior_mean) is list:
        prior_mean = np.array(prior_mean).reshape(-1,1)
    if type(prior_std) is list:
        prior_std = np.array(prior_std).reshape(-1,1)
    if type(data_std) is list:
        data_std = np.array(data_std).reshape(-1,1)
    prior_cov = prior_std*prior_std*np.eye(2) 
    data_cov = data_std*data_std*np.eye(len(data_std)) 
    
    ASA = A@prior_cov@A.T
    
    precision = np.linalg.inv(ASA + data_cov)
    kahlman_update = (prior_cov@A.T@precision)
    post_mean = prior_mean + kahlman_update@(data - A@prior_mean)
    post_cov = prior_cov - kahlman_update@A@prior_cov
    
    return prior_mean, prior_cov, post_mean, post_cov


In [22]:
# THIS EXAMPLE HAS FIXED INPUT AND OUTPUT DIMENSION (2 to 1)

def solve_problem(num_samples = 1000,
                num_observations = 3,
                alpha = 1, 
                beta = 1, 
                data_val = 0,
                data_std = 0.25,
                prior_std_1 = 1,
                prior_std_2 = 1,
                seed=1):
    prior_mean = 0
    prior_std = np.array([prior_std_1, prior_std_2])
    ## define distributions
    
    initial_dist = sstats.distributions.norm(loc=np.zeros(2), scale=prior_std)
    observed_dist = sstats.distributions.norm()
    np.random.seed(seed)
    ## define map and data
    A = linmat(alpha, beta, M=num_observations)
    # instead of defining lambda_true and propagating it, we will just pick a fixed datum and hit it with noise
    noise = np.random.randn(num_observations).reshape(1,-1)
    np.random.seed(seed+3)
    data = data_val + noise

    # generate samples
    input_samples = initial_dist.rvs((num_samples,2))
    output_samples = input_samples@A.T # SIZE = num_samples x num_observations

    def misfit(samples, data, std): # 1-D misfit
        return (1./std)*(1./np.sqrt(len(data)))*np.sum( samples - data, axis=1).reshape(-1,1)

    misfit_samples = misfit(output_samples, data, data_std)

    # plt.hist(misfit_samples)
    # plt.show()
    # xx = np.linspace(-50,50,1000)
    # yy = pf_pdf(xx)
    # plt.plot(xx,yy)

    ## solve inverse problem
    parametric_fit = True
    if parametric_fit:
        l,s = sstats.distributions.norm.fit(misfit_samples.T)
        def pf_pdf(output_samples):
            return sstats.distributions.norm.pdf(output_samples, loc=l, scale=s)
    else:
        def pf_pdf(output_samples):
            return pf_dist.evaluate(output_samples.T).T

    def eval_pdf(input_samples): # evaluate anywhere (for plotting)
        output_samples = input_samples@A.T
        output_samples = misfit(output_samples,data,data_std)
        pushforward_eval = pf_pdf(output_samples) # this is defined in terms of the solution on the original sample set
        intial_eval = np.product(initial_dist.pdf(input_samples),axis=1).reshape(-1,1) # product because 2D independent marginals
        observed_eval = observed_dist.pdf(output_samples)
        ratio = np.divide(observed_eval, pushforward_eval)
        updated_eval = intial_eval*ratio
        return updated_eval

    # Kahlman 
    prior_mean, prior_cov, post_mean, post_cov = gauss_sol(prior_mean, prior_std.T, np.ones(num_observations).T*data_std, A, data.T)
    print(post_mean.T)
    def post_pdf(input_samples):
        return sstats.multivariate_normal.pdf(input_samples, mean=post_mean.ravel(), cov=post_cov, allow_singular=True)


    # updated_eval = eval_pdf(input_samples)
    # posterior_pdf = post_pdf(input_samples)

    n = 100
    x1 = np.linspace(-1, 1, n)
    x2 = x1
    x1, x2 = np.meshgrid(x1,x2)
    xx1, xx2 = x1.ravel(),x2.ravel()
    XX = np.concatenate([xx1, xx2]).reshape(2,n*n).T
    ss = eval_pdf(XX).reshape(n,n).ravel()
    kk = post_pdf(XX).reshape(n,n).ravel()
    s_ind = np.argmax(ss)
    k_ind = np.argmax(kk)
    plt.figure(figsize=(10,10))
    plt.tricontourf(xx1,xx2,ss, 10)
    plt.tricontour(xx1,xx2,kk, 10, cmap='Greys')
    s = (XX[s_ind])@A.T
    k = (XX[k_ind])@A.T
    km = (post_mean.T)@A.T
    print('K :', np.mean(np.power(k[0] - data,2)))
    print('Km:', np.mean(np.power(km[0] - data,2)))
    print('S :', np.mean(np.power(s[0] - data,2)))
#     plt.scatter()
    plt.show()

In [23]:
wid.interactive(solve_problem, 
                num_samples = wid.IntSlider(value=5000, min=500, max=10000, step=100, continuous_update=False),
                num_observations = wid.IntSlider(value=1, min=1, max=500, continuous_update=False),
                alpha = wid.FloatSlider(value=0.5, min=-1, max=1, step=0.05, continuous_update=False),
                beta = wid.FloatSlider(value=-0.25, min=-1, max=1, step=0.05, continuous_update=False),
                data_val = wid.FloatSlider(value=0, min=-1, max=1, step=0.05, continuous_update=False),
                data_std = wid.FloatSlider(value=0.25, min=0.01, max=0.25, step=0.01, continuous_update=False),
                prior_std_1 = wid.FloatSlider(value=0.5, min=0.025, max=2.5, step=0.025, continuous_update=False),
                prior_std_2 = wid.FloatSlider(value=0.5, min=0.025, max=2.5, step=0.025, continuous_update=False),
                seed = wid.IntSlider(value=500, min=1, max=1000, step=1, continuous_update=False)
               )

interactive(children=(IntSlider(value=5000, continuous_update=False, description='num_samples', max=10000, min…